In [1]:
import time as t
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.3.0
Sklearn  0.24.2


In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [3]:
df_train

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [4]:
#missing data
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

pd.set_option('display.max_rows', None)

missing_data

,Total,Percent
PoolQC,1453,0.995205
MiscFeature,1406,0.963014
Alley,1369,0.937671
Fence,1179,0.807534
FireplaceQu,690,0.472603
LotFrontage,259,0.177397
GarageYrBlt,81,0.055479
GarageCond,81,0.055479
GarageType,81,0.055479
GarageFinish,81,0.055479


In [5]:
df_train =df_train.drop((missing_data[missing_data['Total'] > 100]).index,1)
df_train =df_train.drop(df_train.loc[df_train['Electrical'].isnull()].index)
df_train.isnull().sum().max() #just checking that there's no missing data missing...

C:\Users\joshb\AppData\Local\Temp/ipykernel_10596/4064775522.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_train =df_train.drop((missing_data[missing_data['Total'] > 100]).index,1)


81

In [6]:
 X = df_train.drop(['SalePrice','Id'], axis=1)
 y= df_train['SalePrice']
 X_test = df_test.drop(['Id'], axis=1)
 X_test_id = df_test['Id']

In [7]:
X['MSSubClass'] = X['MSSubClass'].astype(str)
X_test['MSSubClass'] = X_test['MSSubClass'].astype(str)

In [8]:
#Split Numerical and Categorical columns

num_cols, cat_cols = [], []

for col in X.columns:
    if X[col].dtype == object:
        cat_cols.append(col)
    else:
        num_cols.append(col)

print('There are', len(cat_cols), 'categorical columns.')
print('There are', len(num_cols), 'numerical columns.')

There are 39 categorical columns.
There are 34 numerical columns.


In [9]:
#cat_vars  = ['MSZoning','Neighborhood']       # x.select_dtypes(include=[object]).columns.values.tolist()
#num_vars  = ['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']

cat_vars = cat_cols
num_vars = num_cols

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']

Categorical features:
 ['MSSubClass', 'MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'Ga

In [10]:
# YOUR CODE HERE
num_4_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(strategy='mean', add_indicator=False))
])

cat_4_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
  ('ordinal', preprocessing.OrdinalEncoder(categories='auto', handle_unknown='use_encoded_value', unknown_value=99999999)) 
  # ('ordinal', preprocessing.OrdinalEncoder(categories='auto', handle_unknown='ignore'))
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars),
    ('cat', cat_4_treeModels, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['LotArea', 'OverallQual', 'OverallCond',
                                  'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
                                  'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
                                  'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
                                  'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
                                  'BsmtHalfBath', 'FullBath', 'HalfBath',
                                  'BedroomAbvGr', 'KitchenAbvGr',
                                  'To...
                                 ['MSSubClass', 'MSZoning', 'Street',
                                  'LotShape', 'LandContour', 'Utilities',
                                  'LotConfig', 'LandSlope', 'Neighborhood',
                                  'Condition1', 'Condition2', 'BldgType',
                                  'HouseStyle', 'RoofStyle', 'RoofMatl',
                                  'Exterior1st', 'Exterior2nd', 'MasVnrType',
                                  'ExterQual', 'ExterCond', 'Foundation',
                                  'BsmtQual', 'BsmtCond', 'BsmtExposure',
                                  'BsmtFinType1', 'BsmtFinType2', 'Heating',
                                  'HeatingQC', 'CentralAir', 'Electrical', ...])])

In [11]:
num_4_multi_models = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(strategy='mean', add_indicator=False)),
  ('scaler',  preprocessing.QuantileTransformer())
])

cat_4_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
  ('onehot', preprocessing.OneHotEncoder(categories='auto', handle_unknown='ignore')) 
]) #handle_unknown='use_encoded_value', unknown_value=99999999 --> what is this ?

multi_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_multi_models, num_vars),
    ('cat', cat_4_treeModels, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

multi_prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler',
                                                  QuantileTransformer())]),
                                 ['LotArea', 'OverallQual', 'OverallCond',
                                  'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
                                  'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
                                  'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
                                  'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
                                  'BsmtHalfBath', 'FullBath', 'HalfBath',...
                                 ['MSSubClass', 'MSZoning', 'Street',
                                  'LotShape', 'LandContour', 'Utilities',
                                  'LotConfig', 'LandSlope', 'Neighborhood',
                                  'Condition1', 'Condition2', 'BldgType',
                                  'HouseStyle', 'RoofStyle', 'RoofMatl',
                                  'Exterior1st', 'Exterior2nd', 'MasVnrType',
                                  'ExterQual', 'ExterCond', 'Foundation',
                                  'BsmtQual', 'BsmtCond', 'BsmtExposure',
                                  'BsmtFinType1', 'BsmtFinType2', 'Heating',
                                  'HeatingQC', 'CentralAir', 'Electrical', ...])])

In [12]:
from sklearn.tree          import DecisionTreeRegressor
from sklearn.ensemble      import RandomForestRegressor
from sklearn.ensemble      import ExtraTreesRegressor
from sklearn.ensemble      import AdaBoostRegressor
from sklearn.ensemble      import GradientBoostingRegressor
from catboost import CatBoostRegressor

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import BayesianRidge

In [14]:

# YOUR CODE HERE
#we dont need scaling/norm for these models
#ordinal encoding (sometimes binary) for categorical 
tree_regressor = {
  "Decision Tree": DecisionTreeRegressor(),
  "Extra Trees":ExtraTreesRegressor(),
  "Random Forest":RandomForestRegressor(),
  "AdaBoost":AdaBoostRegressor(),
  "GBM_regressor":GradientBoostingRegressor(),
  #"HGB_regressor":HistGradientBoostingRegressor(),
  "CatBoost":CatBoostRegressor()
}

#we need to do scaling/norm for these models
#one hot encoding for categorical
mult_regressor= {
    "Linear_regression": LinearRegression(),
    "Ridge_regessor": Ridge(),
    "SVM_regressor":SVR(),
    "MLP_regressor":MLPRegressor(),
    "SDG_regressor": SGDRegressor(),
    "KNN_regressor": KNeighborsRegressor(),
    #"Br_regressor": BayesianRidge(),
}


tree_models_regressor = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_regressor.items()}
multi_models_regressor = {name: pipeline.make_pipeline(multi_prepro, model) for name, model in mult_regressor.items()}


tree_models_regressor["Decision Tree"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'LowQualFinSF', 'GrLivArea',
                                                   'BsmtFullBath',
                                                   'BsmtHalfBath', 'FullBath',
                                                   'HalfBa...
                                                   'LotConfig', 'LandSlope',
                                                   'Neighborhood', 'Condition1',
                                                   'Condition2', 'BldgType',
                                                   'HouseStyle', 'RoofStyle',
                                                   'RoofMatl', 'Exterior1st',
                                                   'Exterior2nd', 'MasVnrType',
                                                   'ExterQual', 'ExterCond',
                                                   'Foundation', 'BsmtQual',
                                                   'BsmtCond', 'BsmtExposure',
                                                   'BsmtFinType1',
                                                   'BsmtFinType2', 'Heating',
                                                   'HeatingQC', 'CentralAir',
                                                   'Electrical', ...])])),
                ('decisiontreeregressor', DecisionTreeRegressor())])

In [15]:
x_train, x_valid, y_train, y_valid = model_selection.train_test_split(X, y, test_size=0.2, random_state=0)

results = pd.DataFrame({'Model': [], 'MSE': [], 'RMSE': [],'RMSLE': [], 'Time': [],})

for name, pipe in tree_models_regressor.items():

    start_time = t.time()

    preds = pipe.fit(x_train,y_train).predict(x_valid)

    total_time = t.time() - start_time

    results = results.append({"Model": name,
                              "MSE":   metrics.mean_squared_error(y_valid, preds),
                              "RMSE":  metrics.mean_squared_error(y_valid, preds,squared=False),
                              'RMSLE': metrics.mean_squared_log_error(y_valid, abs(preds)),
                              "Time":  total_time},
                              ignore_index=True)


    results_ord = results.sort_values(by=['RMSLE'], ascending=True, ignore_index=True)
    results_ord.index += 1 
    clear_output()
    display(results_ord.style.bar(subset=['MSE', 'RMSE','RMSLE'], vmin=0, color='#5fba7d'))

,Model,MSE,RMSE,RMSLE,Time
1,CatBoost,572606540.232754,23929.198487,0.016208,9.765752
2,GBM_regressor,680492185.399502,26086.245138,0.019300,0.723067
3,Extra Trees,743943999.815459,27275.336841,0.021292,1.820863
4,Random Forest,690308973.507496,26273.731625,0.021860,2.120024
5,Decision Tree,1493404663.154109,38644.594229,0.040022,0.076859
6,AdaBoost,1411102730.751399,37564.647353,0.048464,0.389956


In [16]:
# Apply linear regression
x_train, x_valid, y_train, y_valid = model_selection.train_test_split(X, y, test_size=0.2, random_state=0)

results = pd.DataFrame({'Model': [], 'MSE': [], 'RMSE': [],'RMSLE': [], 'Time': [],})

for name, pipe in multi_models_regressor.items():

    start_time = t.time()

    preds = pipe.fit(x_train,y_train).predict(x_valid)

    total_time = t.time() - start_time

    results = results.append({"Model": name,
                              "MSE":   metrics.mean_squared_error(y_valid, preds),
                              "RMSE":  metrics.mean_squared_error(y_valid, preds,squared=False),
                              'RMSLE': metrics.mean_squared_log_error(y_valid, abs(preds)),
                              "Time":  total_time},
                              ignore_index=True)


    results_ord = results.sort_values(by=['RMSLE'], ascending=True, ignore_index=True)
    results_ord.index += 1 
    clear_output()
    display(results_ord.style.bar(subset=['MSE', 'RMSE','RMSLE'], vmin=0, color='#5fba7d'))

,Model,MSE,RMSE,RMSLE,Time
1,SDG_regressor,812571019.123991,28505.631358,0.024927,0.143987
2,Ridge_regessor,858728035.379992,29304.061756,0.028364,0.120975
3,KNN_regressor,1243899826.672055,35268.964071,0.031331,0.152592
4,Linear_regression,1022400163.769393,31974.992788,0.039768,0.278254
5,SVM_regressor,6269067705.137547,79177.444422,0.164569,0.803850
6,MLP_regressor,34192137810.110157,184911.161940,7.949705,4.908871
